### Trying PyDeconv on simulations

In [98]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import mne
from pydeconv.utils import *
from pydeconv.pydeconv_sims import *
from pydeconv import *
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
%matplotlib qt 


# raw     = mne.io.read_raw_eeglab(data_path + "629959_analysis.set", preload=True)

# # Initialize the model

# rERP_model = PyDeconv(settings = settings , features = features, eeg = raw)
# X_design = rERP_model.create_matrix()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
sig = EEGSimulator(200, 500)
W_matrix = [[1]]
kernels = {
            0: {'onsets': [0, 0.19, 0.25], 'amplitudes': [0.1, -0.05, 0.04], 'widths': [0.05, 0.05, 0.07], 'weight':0.3},
            'modulation': {'ker_idx2mod': 1, 'mod': 'linear','dist': 'uniform', 'lims': [100, 600]}
        }

sig.create_isi_pdf(0, sample_size=100, lims=[.3, 1], dist_type='skewed', mode=.50, skew=0, scale=.5)


# sig = EEGSimulator(200, 500)
# W_matrix = [[0, 0.45, 0.45, 0.1],[0.9, 0, 0 , .1],[0.9, 0, 0,.1], [.33,.33,.33,0]]
# kernels = {
#             0: {'onsets': [0, 0.19, 0.25], 'amplitudes': [0.1, -0.05, 0.04], 'widths': [0.05, 0.05, 0.07], 'weight':0.3},
#             1: {'onsets': [0, 0.19, 0.25], 'amplitudes': [0.1, -0.05, 0.04], 'widths': [0.05, 0.05, 0.07], 'weight':0.3},
#             2: {'onsets': [0, 0.19, 0.25], 'amplitudes': [0.1, -0.07, 0.04], 'widths': [0.05, 0.05, 0.07], 'weight':0.3},
#             3: {'onsets': [0, 0.19, 0.25], 'amplitudes': [0.1, -0.07, 0.04], 'widths': [0.05, 0.05, 0.07], 'weight':0.3},
#             'modulation': {'ker_idx2mod': 1, 'mod': 'linear','dist': 'uniform', 'lims': [100, 600]}
#         }

# sig.create_isi_pdf(0, sample_size=100, lims=[.01, .15], dist_type='skewed', mode=.05, skew=0, scale=.01)
# sig.create_isi_pdf(1, sample_size=100, lims=[.1, .6], dist_type='skewed', mode=.3, skew=2, scale=.05)
# sig.create_isi_pdf(2, sample_size=100, lims=[.1, .6], dist_type='skewed', mode=.3, skew=2, scale=.05)
# sig.create_isi_pdf(3, sample_size=100, lims=[.1, .6], dist_type='uniform')


# sig.combine_isi_pdf
# sig.plot_isi_pdf(0)
# sig.plot_isi_pdf(1)


sig.simulate(noise=None,erp_ker=kernels,w_matrix=W_matrix)
sig.plot_response_idx(0)
sig.plot_response_idx(1)
sig.plot_response_idx(2)
plt.show()
sig.plot_datanpsd()
sig.data_stats()
sig.evts['latencies'].diff().hist(bins=50)
plt.show()

    

No noise added
None ISI params for kernel 1
None ISI params for kernel 2
Mean:    0.2931436636599429 
Median:    0.1416908790905415
Variance:  0.23607200761905123


In [154]:
import numpy as np

# Copy and modify the event data
evts = sig.evts.copy()
# evts['effect'] = evts['type'].apply(lambda x: True if x == 2 else False if x == 1 else np.nan)
# evts['type'] = evts['type'].replace(2, 1)
evts['latencies'] = evts['latencies'] * 1000  # Convert latencies to ms
event_id = 0

# Filter events where `type == 1`
filtered_evts = evts.loc[evts['type'] == event_id]

# Get the number of filtered events
n_events = len(filtered_evts)

# Create the events array for MNE
# Column 1: Latencies
# Column 2: Zeros (previous values are assumed 0)
# Column 3: Event types (all set to 1)
mne_events = np.column_stack((filtered_evts['latencies'].values,
                              np.zeros(n_events),
                              filtered_evts['type'].values))

# Convert to integer type
mne_events = mne_events.astype(int)

# Print or use `mne_events` as needed
print(mne_events)


[[     0      0      0]
 [   261      0      0]
 [   606      0      0]
 ...
 [354733      0      0]
 [356663      0      0]
 [357108      0      0]]


In [155]:
import mne

# Parameters
n_seconds = 200        # Duration of the signal in seconds
sfreq = 500            # Sampling frequency in Hz
n_samples = n_seconds * sfreq  # Total number of samples
n_channels = 1         # Number of channels (virtual channel)

# Create random data for the virtual channel (shape: [n_samples])
data = sig.data

# Reshape the data to be 2D (n_channels, n_samples)
data = data.reshape((n_channels, n_samples))

# Create MNE info object
ch_names = ['VirtualEEG']  # Name of the virtual channel
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')

# Create the Raw object from the reshaped data array
raw = mne.io.RawArray(data, info)




Creating RawArray with float64 data, n_channels=1, n_times=100000
    Range : 0 ... 99999 =      0.000 ...   199.998 secs
Ready.


In [168]:
epochs = mne.Epochs(raw, mne_events,  tmin=-0.3, tmax=0.7, preload=True, event_repeated='drop')


Multiple event values for single event times found. Keeping the first occurrence and dropping all others.
Not setting metadata
1199 matching events found
Setting baseline interval to [-0.3, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1199 events and 501 original time points ...
883 bad epochs dropped


In [164]:
mne_events

array([[     0,      0,      0],
       [   261,      0,      0],
       [   606,      0,      0],
       ...,
       [354733,      0,      0],
       [356663,      0,      0],
       [357108,      0,      0]])

In [165]:
evts

,latency,type,categorical,continuous,latencies
0,NaN,0,NaN,NaN,0.000000
1,NaN,0,NaN,NaN,261.449211
2,NaN,0,NaN,NaN,606.059927
3,NaN,0,NaN,NaN,1184.828958
4,NaN,0,NaN,NaN,1533.150790
...,...,...,...,...,...
1195,NaN,0,NaN,NaN,353526.978243
1196,NaN,0,NaN,NaN,354627.283987
1197,NaN,0,NaN,NaN,354733.824472
1198,NaN,0,NaN,NaN,356663.969573


In [169]:
evk = epochs.average()

In [170]:
evk.plot_joint()

No projector specified for this dataset. Please consider the method self.add_proj.
Need more than one channel to make topography for eeg. Disabling interactivity.


/var/folders/gf/gp61rxzn3pz1rqw464l71d7w0000gn/T/ipykernel_63476/979325546.py:1: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  evk.plot_joint()


RuntimeError: No digitization points found.

In [107]:
raw

<RawArray | 1 x 100000 (200.0 s), ~788 kB, data loaded>

In [153]:
evts

,latency,type,categorical,continuous,latencies,effect
0,NaN,0,NaN,NaN,0.000000,True
1,NaN,0,NaN,NaN,497.984134,True
2,NaN,0,NaN,NaN,413.291188,True
3,NaN,0,NaN,NaN,2137.176544,True
4,NaN,0,NaN,NaN,2080.280541,True
...,...,...,...,...,...,...
1195,NaN,0,NaN,NaN,343323.884835,True
1196,NaN,0,NaN,NaN,343240.035109,True
1197,NaN,0,NaN,NaN,343648.205388,True
1198,NaN,0,NaN,NaN,344465.841064,True


In [175]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
n_samples_per_epoch = 1000  # Number of samples per epoch
sfreq = 500  # Sampling frequency (Hz)

# EEG data (replace with actual data from sig object)
data = sig.data  # Assuming data is a NumPy array (could be 1D or 2D)

# Ensure data is 2D (shape: [n_channels, n_samples])
if data.ndim == 1:
    data = data[np.newaxis, :]  # Convert to shape (1, n_samples) if 1D

n_channels, n_samples = data.shape

# Latencies (in sample indices, from mne_events)
latencies_samples = mne_events[:, 0].astype(int)  # Convert latencies to integer indices

# Prepare to store epochs
epochs = []

# Extract epochs (1000 samples per epoch starting at each latency)
for latency in latencies_samples:
    # Make sure we don't go out of bounds
    if latency + n_samples_per_epoch <= n_samples:
        # Extract epoch data for all channels
        epoch = data[:, latency:latency + n_samples_per_epoch]
        epochs.append(epoch)

# Convert epochs list to a NumPy array (shape: [n_epochs, n_channels, n_samples_per_epoch])
epochs = np.array(epochs)

# Average across epochs to get ERP (shape: [n_channels, n_samples_per_epoch])
erp = np.mean(epochs, axis=0)

# Plot the ERP (time-locked averaged signal for the first channel)
time = np.arange(n_samples_per_epoch) / sfreq * 1000  # Time in milliseconds
plt.plot(time, erp[0, :])  # Plotting the ERP for the first channel
plt.xlabel('Time (ms)')
plt.ylabel('Amplitude (µV)')
plt.title('ERP Response - Channel 1')
plt.show()


array([     0,    261,    606, ..., 354733, 356663, 357108])